<div style="background: #000;
            color: #FFF;
            margin: 0px;
                padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 9 - Class 2 - 11/10</h1>
</div>

## Today's agenda:
* Creating template/static folders
* Setting up config Files

# Homework Presentation

### Exercise 2:

Given an int `n`, create a function that creates an `n` x `n` matrix containing increasing integers starting with 1 and write the matrix into a csv file. 


Example:
```    
Input: 1
Output: output.csv
```
The `output.csv` file should contain:
```
1
```
Example:
```    
Input: 2
Output: output.csv
```
The `output.csv` file should contain:

```
1,2
3,4
```


Example:
```    
Input: 3
Output: output.csv
```
The `output.csv` file should contain:

```
1,2,3
4,5,6
7,8,9
```

# More on Flask app config.

[list of config variables](https://flask.palletsprojects.com/en/1.1.x/config/)

A `config` is essentially a dictionary and can be accessed like one.

```python
app = Flask(__name__)
app.config['TESTING'] = True

```

For certain values, you're accessible as attributes:

```python
app = Flask(__name__)
app.testing = True

```

Like dictionaries, we can use the `update` method:

```python
app.config.update(
    TESTING=True,
    SECRET_KEY=b'_5#y2L"F4Q8z\n\xec]/'
)

```

## How do you handle configs when deploying?

### Passing them as environmental variables:

```
$ export SECRET_KEY='5f352379324c22463451387a0aec5d2f'
$ export MAIL_ENABLED=false
$ python run-app.py
 * Running on http://127.0.0.1:5000/
 * Restarting with reloader...
```
which would require the following python code:

```python
import os

_mail_enabled = os.environ.get("MAIL_ENABLED", default="true")
MAIL_ENABLED = _mail_enabled.lower() in {"1", "t", "true"}

SECRET_KEY = os.environ.get("SECRET_KEY")

if not SECRET_KEY:
    raise ValueError("No SECRET_KEY set for Flask application")

```

### Passing them as an object:
```python
app = Flask(__name__)
app.config.from_object('configmodule.ProductionConfig')
```
where the code for `ProductionConfig` looks as follows:
```python
class Config(object):
    DEBUG = False
    TESTING = False
    DATABASE_URI = 'sqlite:///:memory:'

class ProductionConfig(Config):
    DATABASE_URI = 'mysql://user@localhost/foo'

class DevelopmentConfig(Config):
    DEBUG = True

class TestingConfig(Config):
    TESTING = True
```

Keep in mind that values in the config object must be accesible as attributes so use the `@property` decorator.

```python
class Config(object):
    """Base config, uses staging database server."""
    DEBUG = False
    TESTING = False
    DB_SERVER = '192.168.1.56'

    @property
    def DATABASE_URI(self):         # Note: all caps
        return 'mysql://user@{}/foo'.format(self.DB_SERVER)

class ProductionConfig(Config):
    """Uses production database server."""
    DB_SERVER = '192.168.19.32'

class DevelopmentConfig(Config):
    DB_SERVER = 'localhost'
    DEBUG = True

class TestingConfig(Config):
    DB_SERVER = 'localhost'
    DEBUG = True
    DATABASE_URI = 'sqlite:///:memory:'
```

### Common config pattern:
```python
app = Flask(__name__)
app.config.from_object('yourapplication.default_settings')
app.config.from_envvar('YOURAPPLICATION_SETTINGS')

```

This will check for a python object `default_settings`.
Then will check for a file set by the environmental variable called `YOURAPPLICATION_SETTINGS`. If the environmental variable is missing, it defaults to the python object.

You'd set that environmental variable like so:
(in your terminal)
```bash
$ export YOURAPPLICATION_SETTINGS=/path/to/settings.cfg
$ python run-app.py
 * Running on http://127.0.0.1:5000/
 * Restarting with reloader...

```

note: on windows, you'd use the following command:
```
$ set YOURAPPLICATION_SETTINGS=/path/to/settings.cfg
$ python run-app.py
 * Running on http://127.0.0.1:5000/
 * Restarting with reloader...

```

Sample settings.cfg:

```
# Example configuration
DEBUG = False
SECRET_KEY = b'_5#y2L"F4Q8z\n\xec]/'

```

# More on `jinja` templating:

[flask docs on jinja](https://flask.palletsprojects.com/en/1.1.x/templating/)
[jinja docs](https://jinja.palletsprojects.com/en/2.11.x/templates/#if-expression)

global variables available in jinja:
* config - current config.object
* request - current request object
* session - current session object
* g - request-bound object for global variables (requires active context)
* url_for (flask.url_for method)
* get_flashed_messages - gets messages flashed by flask app

types of delimiters:
```
{% ... %} for Statements/Control Structure

{{ ... }} for Expressions to print to the template output

{# ... #} for Comments not included in the template output

#  ... ## for Line Statements
```
[list of functions available for expressions](https://jinja.palletsprojects.com/en/2.11.x/templates/#expressions)


example jinja template:
```html
<!doctype html>
<html>
  <head>
    {# this is a comment #}
    <title>{% block title %}{% endblock %}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>
    {% set navigation = [('index.html', 'Index'), ('about.html', 'About')] %}
    {% set navigation %}
        <li><a href="/">Index</a>
        <li><a href="/downloads">Downloads</a>
    {% endset %}
    {% set key, value = call_something() %}
    <header>
      {% block header %}{% endblock %}
    </header>
    <nav>
      <ul>
         {% for link in nav_links if not link.hidden %}
          <li><a href="{{ link.href }}">{{ link.title }}</a></li>
          {% else %}
           {{ navigation }}
         {% endfor %}
      </ul>
    </nav>
    <section class="content">
    {% block content %}{% endblock %}
        
    {{ data |tojson }}
    
    
    {%  autoescape false %}
        {{ will_not_be_escaped }}
    {% autoescape % }
        
    
    {% include 'footer.html' %}
        
    </section>
    <ul class="sitemap">
    {%- for item in sitemap recursive %}
        <li><a href="{{ item.href|e }}">{{ item.title }}</a>
        {%- if item.children -%}
            <ul class="submenu">{{ loop(item.children) }}</ul>
        {%- endif %}</li>
    {%- endfor %}
</ul>
</body>


```
### Filters

`tojson()` is a ilter method that turns a python object to json
other filters:
* safe - renders string as safe html

creating your own:
```python
@app.template_filter('reverse')
def reverse_filter(s):
    return s[::-1]


def reverse_filter(s):
    return s[::-1]
app.jinja_env.filters['reverse'] = reverse_filter
```





# More on defining routes:

We've seen how to define routes using the `@app.route()` decorator.

We can also create them like so:

```python
from flask import Flask

def home():
    return render_template("home.html")

app.add_url_rule('/', 'home', home)

```

Notes:
    
Adding a trailing slash means a website is accessible with or without a trailing slash

```python

@app.route('/projects/')
def projects():
    return render_template("projects.html")

@app.route('/about')
def about():
    return render_template("about.html")
```
projects is accessible from:
    127.0.0.1:5000/projects
    127.0.0.1:5000/projects/
    
about is accessible from:
    127.0.0.1:5000/about

We can pass variables to our routes:

```python
@app.route('/user/<username>')
def show_user_profile(username):
    return render_template("user.html", user=username)

```

Adding a default argument can be done like so:
```python
@app.route('/user/')
@app.route('/user/<username>')
def show_user_profile(username=None):
    return render_template("user.html", user=username)
```

Changing accepted HTTP Methods

```python
from flask import request

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        return do_the_login()
    else:
        return show_the_login_form()

```

    
    

# Setting up a Flask project:
   
The basic structure of your flask application will be:
```
project-folder/
    /static
        style.css
    /templates
        base.html
        home.html
    app.py
```


#### The `static.css` file:

```css
html, body {
    margin: 0;
    padding: 0;
    width: 100vw;
    height:100vh;
    background: #FFF; /* your choice */
    color: #000; /*your choice */
}
```

#### The `base.html` file

```html
<!doctype html>
<html>
  <head>
    <title>{% block title %}{% endblock %}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>
    <header>
      {% block header %}{% endblock %}
    </header>
    <nav>
      <ul>
        <li><a href="{{ url_for('home') }}">home</a></li>
      </ul>
    </nav>
    <section class="content">
    {% block content %}{% endblock %}
    </section>
</body>
```

As you can see, there are `{% block block_name %} {% endblock %}` tags that represent areas where the child html file will replace with information.

The `{{ statement }}` tags will output python code similarly to f-strings. In order to pass variables to the template, you must include them as a keyword argument to the `render_template` method which handles passing data from the controller to the template.

The `url_for` method takes a string representing the name of one of our controllers. Flask (using jinja) will replace that with the actually hard link to the page.

#### The `home.html` file

This is leveraging the `base.html` and uses `jinja` as the templating engine to output our python into the html.

example of what could be in your 

```html
{% extends "base.html" %}

{% block title %}home page{% endblock %}

{% block header %}
<h1>header</h1>

{% endblock %}

{% block content %}
<div>
<p>hello<p>
</div>
{% endblock %} 
```

#### The `app.py` file:

This is the basic structure of your project to start off.

```python 

from flask import Flask, render_template, url_for

app = Flask()

@app.route('/')
def home():
    return render_template("home.html")

if __name__ == '__main__':
    app.run(debug=True)

```

# Introduction to Git:

Git is a free and open source distributed version control system designed to handle everything from small to very large projects with speed and efficiency.

Git is easy to learn and has a tiny footprint with lightning fast performance. It outclasses SCM tools like Subversion, CVS, Perforce, and ClearCase with features like cheap local branching, convenient staging areas, and multiple workflows.

[source](https://git-scm.com/)


We're going to use github. Gitlab is a good alternative.

Download Git:
https://git-scm.com/downloads

Setting up Git (according to github):
* Open Terminal
* Set up a Git:  

(globally)
```
$ git config --global user.name "John Doe"
$ git config --global user.email johndoe@example.com
```  

(locally)
```
$ git config user.name "John Doe"
$ git config user.email 
```

* Check Git configs:
```
$ git config --global user.name
$ git config --global user.email
$ git config --list
```
(more on git config: https://git-scm.com/docs/git-config)
    


https://docs.github.com/en/free-pro-team@latest/github/getting-started-with-github/set-up-git

More documentation:

[GitHub Flow](https://docs.github.com/en/free-pro-team@latest/github/collaborating-with-issues-and-pull-requests/github-flow)

[Associating text editors with Git](https://docs.github.com/en/free-pro-team@latest/github/using-git/associating-text-editors-with-git)

# Cloning and Pulling from a git repo

basics:

There's usually a master branch 


This command will clone a git repo into your current directory:
```
$ git clone https://github.com/The-Knowledge-House/I2020_DS
```

This command will pull from that git repo when changes are made:
```
$ git pull 
```
To pull a specific branch:

```
$ git pull origin master
```

`git pull` is doing two other git commands:

```
$ git fetch
$ git merge FETCH_HEAD
```

What is fetching?

Fetching will get new information from a repo. 

```
$ git fetch

```
What is merging?

Merging is adding changes from a different commit or branch into your current branch
```
$ git merge
```

You get merge conflicts when you pull because you fetch new changes to the repo into your local repo and then git doesn't know how to handle the overlap. Basically the conflicts are where the new changes and your code conflict and git expects you to "fix it" by choosing which changes to keep.

[article on merge conflicts](https://www.atlassian.com/git/tutorials/using-branches/merge-conflicts)

If you'd like to avoid having to deal with merge conflicts, you can use the following flow:
* stash your local changes away using `git stash`
* use `git pull` to fetch and merge in changes
* apply your stashed changes to the repo after merge using `git stash apply`

# Committing your changes

A git commit will record changes in your repo. You're then able to push them to the main repo when you're ready.

To commit, you need to do the folllowing steps:
* add changes to the index using `git add`
* commit those changes with a message.
* pushing the commit (in your case to the 'master branch')

```
$ git add <files>
$ git commit -m "this is where you explain what changed"
$ git push origin master
```

